In [7]:
import pandas as pd
import numpy as np
import torch
import os
filepath = 'dataLink/train/'
!pip3 install ultralytics
import ultralytics
ultralytics.checks()

In [5]:
from ultralytics import YOLO
model = YOLO('yolov8m.pt') 

Ultralytics YOLOv8.1.16 🚀 Python-3.9.18 torch-2.2.0+cu121 CPU (AMD EPYC 7V13 64-Core Processor)
Setup complete ✅ (256 CPUs, 251.2 GB RAM, 63.3/70.0 GB disk)


In [6]:
 # load a pretrained model (recommended for training)
# model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights

100%|██████████| 49.7M/49.7M [00:00<00:00, 368MB/s]


In [8]:
filepath = 'datasets/train/labels.txt'
df = pd.read_csv(filepath, sep=' ', header=None)

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/train/labels.txt'

In [10]:
# Correcting the adjustment and applying the correct divisions
df[2] = df[2] / 960 
df[4] = df[4] / 960 
df[3] = df[3] / 540 
df[5] = df[5] / 540 

In [12]:
# Define the output directory for demonstration purposes
output_dir = "datasets/train/labels/"

# Determine the maximum length for padding
max_length = len(str(df[0].max()))

# Process and prepare to save files for a subset of unique identifiers, excluding the first column
for value in df[0].unique():  
    # Filter rows for the current unique value and exclude the first column
    subset_df = df[df[0] == value].iloc[:, 1:]
    subset_df.iloc[:, 0] = subset_df.iloc[:, 0] - 1
    
    file_name = f"{str(value).zfill(max_length)}.txt"
    
    file_path = os.path.join(output_dir, file_name)
    subset_df.to_csv(file_path, header=False, index=False, sep=' ')


In [13]:
filepath_val = 'datasets/val/labels.txt'
df_val = pd.read_csv(filepath_val, sep=' ', header=None)

In [15]:
# Correcting the adjustment and applying the correct divisions
df_val[2] = df_val[2] / 960 
df_val[4] = df_val[4] / 960 
df_val[3] = df_val[3] / 540 
df_val[5] = df_val[5] / 540 

In [18]:
# Define the output directory for demonstration purposes
output_dir = "datasets/val/labels/"
max_length = 5
for value in df_val[0].unique():  
    # Filter rows for the current unique value and exclude the first column
    subset_df = df_val[df_val[0] == value].iloc[:, 1:]
    subset_df.iloc[:, 0] = subset_df.iloc[:, 0] - 1
    
    file_name = f"{str(value).zfill(max_length)}.txt"
    file_path = os.path.join(output_dir, file_name)
    
    subset_df.to_csv(file_path, header=False, index=False, sep=' ')

subset_df_example = df_val[df_val[0] == df_val[0].unique()[0]].iloc[:, 1:]


,1,2,3,4,5
0,1,0.764583,0.316667,0.095833,0.092593


In [16]:
# Freezing model layers for faster training
def freeze_layer(trainer):
    model = trainer.model
    num_freeze = 10
    print(f"Freezing {num_freeze} layers")
    freeze = [f'model.{x}.' for x in range(num_freeze)]  # layers to freeze 
    for k, v in model.named_parameters(): 
        v.requires_grad = True  # train all layers 
        if any(x in k for x in freeze): 
            print(f'freezing {k}') 
            v.requires_grad = False 
    print(f"{num_freeze} layers are freezed.")

In [17]:
model = YOLO('yolov8m.pt')
model.add_callback("on_train_start", freeze_layer)
results = model.train(data='data.yaml', epochs=75, imgsz=640)


Ultralytics YOLOv8.1.16 🚀 Python-3.8.17 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24214MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=75, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=T

train: Scanning /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/labels.cache... 14000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 14000/14000 [00:00<?, ?it/s]

train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/00288.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/00491.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/00787.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/00802.jpeg: 2 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/01505.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/02366.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/02431.jpeg: 1 duplicate labels removed
train: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/train/images/02549.jpeg: 2 duplicate labels 

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/val/labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

...

      75/75      3.85G      1.171     0.5935        1.1         58        640: 100%|██████████| 875/875 [02:03<00:00,  7.08it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:05<00:00, 10.72it/s]

                   all       2000       9061      0.543       0.69      0.637      0.427



75 epochs completed in 2.721 hours.
Optimizer stripped from runs/detect/train27/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train27/weights/best.pt, 52.0MB

Validating runs/detect/train27/weights/best.pt...
Ultralytics YOLOv8.1.16 🚀 Python-3.8.17 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24214MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 63/63 [00:06<00:00,  9.79it/s]


                   all       2000       9061      0.602      0.669      0.663      0.443
                   car       2000       8781      0.791      0.884      0.899      0.577
          medium truck       2000        163      0.505      0.644      0.551      0.379
           large truck       2000        117       0.51      0.479       0.54      0.373
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/train27


In [41]:
data_yaml_path = os.path.abspath('data.yaml')
model = YOLO('runs/detect/train27/weights/best.pt')
result_grid = model.tune(data=data_yaml_path, epochs = 10, imgsz = 640, use_ray=True, gpu_per_trial = 1)

In [39]:
# Load a model
# model = YOLO('yolov8n.pt')  # load an official model
model = YOLO('runs/detect/train27/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.1.16 🚀 Python-3.8.17 torch-2.0.1 CUDA:0 (NVIDIA GeForce RTX 4090, 24214MiB)
Model summary (fused): 218 layers, 25841497 parameters, 0 gradients, 78.7 GFLOPs


val: Scanning /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/val/labels.cache... 2000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2000/2000 [00:00<?, ?it/s]

val: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/val/images/00425.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/val/images/00954.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/val/images/01642.jpeg: 1 duplicate labels removed
val: WARNING ⚠️ /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/val/images/01755.jpeg: 1 duplicate labels removed



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 125/125 [00:07<00:00, 17.84it/s]


                   all       2000       9061      0.604       0.67      0.663      0.443
                   car       2000       8781      0.791      0.883      0.899      0.576
          medium truck       2000        163      0.505      0.644      0.551      0.378
           large truck       2000        117      0.517      0.484       0.54      0.374
Speed: 0.1ms preprocess, 2.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val2


array([    0.57649,     0.37754,     0.37384])

In [37]:
from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train27/weights/best.pt')

# Define path to the image file
source = 'datasets/test/images/'

# Run inference on the source
results = model.predict(source, save_txt = True, save_conf = True, conf = 0.05) 



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/2000 /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/test/images/00001.jpeg: 384x640 23 cars, 6.9ms
image 2/2000 /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/test/images/00002.jpeg: 384x640 3 cars, 6.5ms
image 3/2000 /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/test/images/00003.jpeg: 384x640 16 cars, 6.8ms
image 4/2000 /media/imaginarium/12T_2/A_Work/A_DL/Program_2/datasets/test/images/00004.jpeg: 384x640 8 cars, 

In [33]:
len(results)

2000

In [40]:
import os
import pandas as pd

# Define the input directory where your files are located
input_dir = "runs/detect/predict2/labels"
output_file_path = "datasets/test/labels.txt"

# Automatically find all .txt files in the directory
files = os.listdir(input_dir)
files = [f for f in files if f.endswith('.txt')]

# Initialize a counter
counter = 1

# Initialize an empty list to store dataframes
dfs = []

# Process each file
for file in sorted(files):
    file_path = os.path.join(input_dir, file)
    print(f"Processing file: {file}")  # Print the file name being processed
    
    # Read the file into a DataFrame, assuming space as the delimiter
    df = pd.read_csv(file_path, header=None, sep=' ')
    
    # Increment the first column by 1 to adjust class numbers from 0-2 to 1-3
    df.iloc[:, 0] = df.iloc[:, 0] + 1
    
    # Insert the counter as the first column of the DataFrame
    df.insert(0, 'Counter', counter)
    
    # Append the DataFrame to the list
    dfs.append(df)
    
    # Increment the counter for the next file
    counter += 1

# Combine all DataFrames into a single one
combined_df = pd.concat(dfs, ignore_index=True)
combined_df.to_csv(output_file_path, index=False, header=False, sep=' ')

Processing file: 00001.txt
Processing file: 00002.txt
Processing file: 00003.txt
Processing file: 00004.txt
Processing file: 00005.txt
Processing file: 00006.txt
Processing file: 00007.txt
Processing file: 00008.txt
Processing file: 00009.txt
Processing file: 00010.txt
Processing file: 00011.txt
Processing file: 00012.txt
Processing file: 00013.txt
Processing file: 00014.txt
Processing file: 00015.txt
Processing file: 00016.txt
Processing file: 00017.txt
Processing file: 00018.txt
Processing file: 00019.txt
Processing file: 00020.txt
Processing file: 00021.txt
Processing file: 00022.txt
Processing file: 00023.txt
Processing file: 00024.txt
Processing file: 00025.txt
Processing file: 00026.txt
Processing file: 00027.txt
Processing file: 00028.txt
Processing file: 00029.txt
Processing file: 00030.txt
Processing file: 00031.txt
Processing file: 00032.txt
Processing file: 00033.txt
Processing file: 00034.txt
Processing file: 00035.txt
Processing file: 00036.txt
Processing file: 00037.txt
P